In [8]:
import time
import requests
import urllib
import json
from datetime import datetime
import dateutil.parser
from splinter import Browser
from tosConfig import consumerKey, passWord, userName
from tdAuthen import accesstoken

In [89]:
authen_data = accesstoken(consumerKey = consumerKey, passWord = passWord, userName = userName)

Pulled Code, grabbing access token....


In [87]:
import datetime
def unix_time_millis(dt):
    epoch = datetime.datetime.utcfromtimestamp(0)
    return (dt - epoch).total_seconds() * 1000.0

In [88]:
# define endpoint which is the User Principles emdpoint

endpoint = 'https://api.tdameritrade.com/v1/userprincipals'
headers = {'Authorization':'Bearer {}'.format(authen_data['access_token'])}

# define the paramenters for the endpoint
params = {'fields': 'streamerSubscriptionKeys,streamerConnectionInfo'}

# make a requests
content = requests.get(url = endpoint, params = params, headers = headers)
userPrincipalsResponse = content.json()

# defining items needed to make requests to login
tokenTimeStamp = userPrincipalsResponse['streamerInfo']['tokenTimestamp']
date = dateutil.parser.parse(tokenTimeStamp, ignoretz = True)
tokenTimeStampAsMs = unix_time_millis(date)

credentials = {'userid':userPrincipalsResponse['accounts'][0]['accountId'],
              'token':userPrincipalsResponse['streamerInfo']['token'],
              'company':userPrincipalsResponse['accounts'][0]['company'],
              'segment':userPrincipalsResponse['accounts'][0]['segment'],
              'cddomain':userPrincipalsResponse['accounts'][0]['accountCdDomainId'],
              'usergroup':userPrincipalsResponse['streamerInfo']['userGroup'],
                'accesslevel': userPrincipalsResponse['streamerInfo']['accessLevel'],
              'authorized':'Y',
              'timestamp':int(tokenTimeStampAsMs),
              'appid':userPrincipalsResponse['streamerInfo']['appId'],
              'acl':userPrincipalsResponse['streamerInfo']['acl']}

KeyError: 'streamerInfo'

In [61]:
# defining login request
login_request = {"requests":[{"service": "ADMIN",
                        "command": "LOGIN",
                        "requestid": "0",
                        "account":userPrincipalsResponse['accounts'][0]['accountId'],
                        "source":userPrincipalsResponse['streamerInfo']['appId'],
                        "parameters": {"credential": urllib.parse.urlencode(credentials),
                                        "token":userPrincipalsResponse['streamerInfo']['token'],
                                        "version": "1.0"}}]}

data_request = {"requests": [{"service":"TIMESALE_EQUITY",
                "requestid": "0",
                "command": "SUBS",
                "account":userPrincipalsResponse['accounts'][0]['accountId'],
                "source": userPrincipalsResponse['streamerInfo']['appId'],
                "parameters": {
                "keys": "AAPL",
                "fields": "1,2,3,4"}}]} 

login_encoded = json.dumps(login_request)
data_encoded = json.dumps(data_request)

data_request


{'requests': [{'service': 'TIMESALE_EQUITY',
   'requestid': '0',
   'command': 'SUBS',
   'account': '496550006',
   'source': 'lopezproceso',
   'parameters': {'keys': 'AAPL', 'fields': '1,2,3,4'}}]}

In [85]:
import websockets
import asyncio
import pyodbc
import csv


# This connects to the database
class WebSocketClient():
    
#     def __init__(self):
#         self.data_holder = []
#         self.file = open('td_data.txt', 'a')
#         self.cnxn = None
#         self.crsr = None
        
# # # # defining the csv file to connect to 
# # #     def csv_connect(self):
# # #        csvfile = 'testWrite.csv'
        
# #     def csv_insert(self, data_tuple):
        
# #         csvfile = 'testWrite.csv'
# #         with open(csvfile, 'w') as ouput:
# #             output_data = csv.writer(output, delimiter = ',')
# #             csv.output_date.writerow(data_tuple)
        
# #         print('Data has been successfully inserted into the database')
    

    
    async def connect(self):
        
        uri = "wss://" + userPrincipalsResponse['streamerInfo']['streamerSocketUrl'] + "/ws"
        self.connection = await websockets.client.connect(uri)
        
        if self.connection.open:
            print("Connection established. Client correctly connected.")
            return self.connection
   
    
    async def sendMessage(self, message):
        
        await self.connection.send(message)
        
    async def receiveMessage(self, connection):
        
        while True: 
            try:
                #grab and decode the message
                message = await connection.recv()
                message_decoded = json.loads(message)

                if 'data' in message_decoded.keys():
                    
                    #grab the data
                    data = message_decoded['data'][0]
                    data_tuple = (data['service'], data['timestamp'], data['command'])
                    
                    # insert the data
                    self.csv_insert(data_tuple)
                
                print('-'*20)
                print('Recieved message from server' + str(message))
                
                
            except websockets.exceptions.ConnectionClosed:
                print('Connection with server closed')
                break
    
    async def heartbeat(self, connection):
        while True:
            try: 
                await connection.send('ping')
                await aysncio.sleep(5)
            except websockets.exceptions.ConnectionClosed:
                print('Connection with server closed')
                break
            

In [86]:
import nest_asyncio
nest_asyncio.apply()

if __name__ == '__main__':

    # creat the client object
    client = WebSocketClient()
    
    # define and event loop
    loop = asyncio.get_event_loop()
    
    #start a connection to the websocket
    connection = loop.run_until_complete(client.connect())
    
    #define the tasks that we want to run
    tasks = [asyncio.ensure_future(client.recieveMessage(connection)),
             asyncio.ensure_future(client.sendMessage(login_encoded)),
             asyncio.ensure_future(client.recieveMessage(connection)),
             asyncio.ensure_future(client.sendMessage(data_encoded)),
             asyncio.ensure_future(client.recieveMessage(connection))]
    
    loop.run_until_complete(asyncio.wait(tasks))
    


NegotiationError: Unsupported extension: name = permessage-deflate, params = []